In [1]:
from scapy.all import *


try:
    packets = rdpcap('2.pcap')  
except FileNotFoundError:
    print("Error: PCAP file not found. Make sure it's in the same directory.")
    exit()


dns_queries = [p for p in packets if p.haslayer(DNS) and p[DNS].qr == 0]

print(f"Found {len(dns_queries)} DNS queries in the pcap file.")


if dns_queries:
    first_query = dns_queries[0]
    print("\n--- Original DNS Query (Packet Summary) ---")
    first_query.show()

Found 24 DNS queries in the pcap file.

--- Original DNS Query (Packet Summary) ---
###[ Ethernet ]###
  dst       = 01:00:5e:00:00:fb
  src       = 14:10:9f:cf:e1:71
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 569
     id        = 50734
     flags     = 
     frag      = 0
     ttl       = 255
     proto     = udp
     chksum    = 0xe166
     src       = 172.16.133.18
     dst       = 224.0.0.251
     \options   \
###[ UDP ]###
        sport     = mdns
        dport     = mdns
        len       = 549
        chksum    = 0xd460
###[ DNS ]###
           id        = 0
           qr        = 0
           opcode    = QUERY
           aa        = 0
           tc        = 0
           rd        = 0
           ra        = 0
           z         = 0
           ad        = 0
           cd        = 0
           rcode     = ok
           qdcount   = 14
           ancount   = 6
           nscount   = 0
           arcount   = 0
       

In [ ]:
# print(dns_queries[0])

<bound method Packet.canvas_dump of <Ether  dst=01:00:5e:00:00:fb src=14:10:9f:cf:e1:71 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=569 id=50734 flags= frag=0 ttl=255 proto=udp chksum=0xe166 src=172.16.133.18 dst=224.0.0.251 |<UDP  sport=mdns dport=mdns len=549 chksum=0xd460 |<DNS  id=0 qr=0 opcode=QUERY aa=0 tc=0 rd=0 ra=0 z=0 ad=0 cd=0 rcode=ok qdcount=14 ancount=6 nscount=0 arcount=0 qd=[<DNSQR  qname=b'_apple-mobdev._tcp.local.' qtype=PTR unicastresponse=0 qclass=IN |>, <DNSQR  qname=b'_airplay._tcp.local.' qtype=PTR unicastresponse=0 qclass=IN |>, <DNSQR  qname=b'_raop._tcp.local.' qtype=PTR unicastresponse=0 qclass=IN |>, <DNSQR  qname=b'_odisk._tcp.local.' qtype=PTR unicastresponse=0 qclass=IN |>, <DNSQR  qname=b'_afpovertcp._tcp.local.' qtype=PTR unicastresponse=0 qclass=IN |>, <DNSQR  qname=b'_smb._tcp.local.' qtype=PTR unicastresponse=0 qclass=IN |>, <DNSQR  qname=b'_rfb._tcp.local.' qtype=PTR unicastresponse=0 qclass=IN |>, <DNSQR  qname=b'_adisk._tcp.local.' qtype=PTR unica

In [ ]:
import datetime

class CustomDNSHeader(Packet):
    name = "CustomDNSHeader"
    fields_desc = [
        StrFixedLenField("timestamp_id", "", 8)
    ]


now = datetime.datetime.now()
sequence_id = 0  


modified_packets = []

for query in dns_queries:
    if(len(str(sequence_id))) == 1:
        sequence_id_str = '0' + str(sequence_id)
    else:
        sequence_id_str = str(sequence_id)
        
    header_value_str = now.strftime("%H%M%S") + sequence_id_str
    sequence_id += 1  
    header_value_bytes = header_value_str.encode('utf-8')
    packet_with_header = CustomDNSHeader(timestamp_id=header_value_bytes) / query
    modified_packets.append(packet_with_header)

modified_packets[0].show()

###[ CustomDNSHeader ]###
  timestamp_id= b'22145200'
###[ Ethernet ]###
     dst       = 01:00:5e:00:00:fb
     src       = 14:10:9f:cf:e1:71
     type      = IPv4
###[ IP ]###
        version   = 4
        ihl       = 5
        tos       = 0x0
        len       = 569
        id        = 50734
        flags     = 
        frag      = 0
        ttl       = 255
        proto     = udp
        chksum    = 0xe166
        src       = 172.16.133.18
        dst       = 224.0.0.251
        \options   \
###[ UDP ]###
           sport     = mdns
           dport     = mdns
           len       = 549
           chksum    = 0xd460
###[ DNS ]###
              id        = 0
              qr        = 0
              opcode    = QUERY
              aa        = 0
              tc        = 0
              rd        = 0
              ra        = 0
              z         = 0
              ad        = 0
              cd        = 0
              rcode     = ok
              qdcount   = 14
              an

In [19]:
modified_packets[0].show2()

###[ CustomDNSHeader ]###
  timestamp_id= b'22145200'
###[ Raw ]###
     load      = b'\x01\x00^\x00\x00\xfb\x14\x10\x9f\xcf\xe1q\x08\x00E\x00\x029\xc6.\x00\x00\xff\x11\xe1f\xac\x10\x85\x12\xe0\x00\x00\xfb\x14\xe9\x14\xe9\x02%\xd4`\x00\x00\x00\x00\x00\x0e\x00\x06\x00\x00\x00\x00\r_apple-mobdev\x04_tcp\x05local\x00\x00\x0c\x00\x01\x08_airplay\xc0\x1a\x00\x0c\x00\x01\x05_raop\xc0\x1a\x00\x0c\x00\x01\x06_odisk\xc0\x1a\x00\x0c\x00\x01\x0b_afpovertcp\xc0\x1a\x00\x0c\x00\x01\x04_smb\xc0\x1a\x00\x0c\x00\x01\x04_rfb\xc0\x1a\x00\x0c\x00\x01\x06_adisk\xc0\x1a\x00\x0c\x00\x01\x08_airport\xc0\x1a\x00\x0c\x00\x01\x04_ipp\xc0\x1a\x00\x0c\x00\x01\x08_scanner\xc0\x1a\x00\x0c\x00\x01\x08_printer\xc0\x1a\x00\x0c\x00\x01\x04_ptp\xc0\x1a\x00\x0c\x00\x01\x1eSean Armstrong\xe2\x80\x99s MacBook Pro\xc0E\x00\x10\x00\x01\xc0\x96\x00\x0c\x00\x01\x00\x00\x10\x0b\x00$!HP LaserJet P3010 Series [8E108F]\xc0\x96\xc0\x96\x00\x0c\x00\x01\x00\x00\x10\x0b\x00$!HP LaserJet P3010 Series [0F1041]\xc0\x96\xc0\x96\x00\x0c\x0

In [14]:
modified_packets[1].show2()

###[ CustomDNSHeader ]###
  timestamp_id= b'22113401'
###[ Raw ]###
     load      = b'33\x00\x00\x00\xfb\x14\x10\x9f\xcf\xe1q\x86\xdd`\x00\x00\x00\x02%\x11\xff\xfe\x80\x00\x00\x00\x00\x00\x00\x16\x10\x9f\xff\xfe\xcf\xe1q\xff\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfb\x14\xe9\x14\xe9\x02%Q\xaf\x00\x00\x00\x00\x00\x0e\x00\x06\x00\x00\x00\x00\r_apple-mobdev\x04_tcp\x05local\x00\x00\x0c\x00\x01\x08_airplay\xc0\x1a\x00\x0c\x00\x01\x05_raop\xc0\x1a\x00\x0c\x00\x01\x06_odisk\xc0\x1a\x00\x0c\x00\x01\x0b_afpovertcp\xc0\x1a\x00\x0c\x00\x01\x04_smb\xc0\x1a\x00\x0c\x00\x01\x04_rfb\xc0\x1a\x00\x0c\x00\x01\x06_adisk\xc0\x1a\x00\x0c\x00\x01\x08_airport\xc0\x1a\x00\x0c\x00\x01\x04_ipp\xc0\x1a\x00\x0c\x00\x01\x08_scanner\xc0\x1a\x00\x0c\x00\x01\x08_printer\xc0\x1a\x00\x0c\x00\x01\x04_ptp\xc0\x1a\x00\x0c\x00\x01\x1eSean Armstrong\xe2\x80\x99s MacBook Pro\xc0E\x00\x10\x00\x01\xc0\x96\x00\x0c\x00\x01\x00\x00\x10\x0b\x00$!HP LaserJet P3010 Series [8E108F]\xc0\x96\xc0\x96\x00\x0c\x00\x01\x

In [ ]:
import socket



SERVER_IP = "127.0.0.1" 
SERVER_PORT = 9998

packet_to_send = modified_packets[6]
with open('report.csv', 'w') as f:
    f.write("Header Value,Domain Name,Resolved IP address\n")
    for query in modified_packets:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            print(f"Connecting to server at {SERVER_IP}:{SERVER_PORT}...")
            s.connect((SERVER_IP, SERVER_PORT))


            s.sendall(bytes(query)) 

            response = s.recv(1024)
            print(f"Server responded with: {response.decode()}")
            f.write(f"{query[CustomDNSHeader].timestamp_id.decode()},{query[DNSQR].qname.decode()},{response.decode()}\n")

Connecting to server at 127.0.0.1:9998...
Server responded with: 192.168.1.11
Connecting to server at 127.0.0.1:9998...
Server responded with: 192.168.1.12
Connecting to server at 127.0.0.1:9998...
Server responded with: 192.168.1.13
Connecting to server at 127.0.0.1:9998...
Server responded with: 192.168.1.14
Connecting to server at 127.0.0.1:9998...
Server responded with: 192.168.1.15
Connecting to server at 127.0.0.1:9998...
Server responded with: 192.168.1.11
Connecting to server at 127.0.0.1:9998...
Server responded with: 192.168.1.12
Connecting to server at 127.0.0.1:9998...
Server responded with: 192.168.1.13
Connecting to server at 127.0.0.1:9998...
Server responded with: 192.168.1.14
Connecting to server at 127.0.0.1:9998...
Server responded with: 192.168.1.15
Connecting to server at 127.0.0.1:9998...
Server responded with: 192.168.1.11
Connecting to server at 127.0.0.1:9998...
Server responded with: 192.168.1.12
Connecting to server at 127.0.0.1:9998...
Server responded with: